In [ ]:
# mapping S --> T, generates r random maps 

function randommap(S::String, T::String, r::Int)
    # for each position of the map (each gene w >1 instances)
        # an integer value is selcted uniformly from the interval [0, occ(alpha, S)! -1] 
        # intiially the target string is mapped into permutation Pp using an arbitrary map p 
        # next r RM of src str S are generated and stored in a set M 

    # for each map m in M 
        # distance between permutation Sm and Pp is computed using estimator algo 

    #return shortest map iwth shortest dist between permutations 
end 